In [9]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision import transforms as T
import torch.optim as optim
import torchmetrics

# Для чтения изображений с диска
from torchvision import io # input/output
import torchutils as tu
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [10]:
trnsfrms = T.Compose(
    [
        T.Resize((224, 224)),
        T.RandomHorizontalFlip(),
        T.RandomRotation(23),
        #T.RandomPerspective(),
        T.ToTensor() # автоматически интервал пикселей будет 0-1
    ]
)

In [11]:
train_dataset = torchvision.datasets.ImageFolder(
    './data/archive_sport/train',
    transform=trnsfrms
)
valid_dataset = torchvision.datasets.ImageFolder(
    './data/archive_sport/valid',
    transform=trnsfrms#T.Compose([T.Resize((224, 224)), T.ToTensor()])
)

In [12]:
idx2class= {j: i for i, j in train_dataset.class_to_idx.items()}

In [13]:
labels = idx2class.keys()

In [15]:
test_dataset = TensorDataset(valid_dataset, labels)

AttributeError: 'ImageFolder' object has no attribute 'size'

In [ ]:
test_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

In [11]:
model = torch.load('./models/model_sport/model_sport.pth')

# Перевод модели в режим eval
model.eval()

/tmp/ipykernel_191049/3335324354.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('./models/model_sport/model_sport.pth')


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
def calculate_metrics(outputs, labels):
    predictions = torch.argmax(outputs, dim=1).cpu().numpy()
    labels = labels.cpu().numpy()
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted') # 'weighted' для учета несбалансированных классов
    return accuracy, f1

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

test_accuracy = []
test_f1 = []

with torch.no_grad():
    for data, labels in valid_dataset:
        data = data.to(device)
        labels = labels.to(device)

        outputs = model(data)
        accuracy, f1 = calculate_metrics(outputs, labels)

        test_accuracy.append(accuracy)
        test_f1.append(f1)

# Вывод результатов
print(f'Accuracy: {sum(test_accuracy)/len(test_accuracy):.4f}')
print(f'F1-мера: {sum(test_f1)/len(test_f1):.4f}')

NameError: name 'test_loader' is not defined